<a href="https://colab.research.google.com/github/Feru34/Deep_Learning/blob/main/Taller_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip -q
!pip install keras-tuner -q
!pip install kaggle -q
!pip install scikeras -q
# El '-q' hace referencia a 'quiet'. Por lo tanto el output de los paquetes no se muestra completo

  Preparing metadata (setup.py) ... done


In [2]:
import numpy as np
import pandas as pd
from google.colab import files

from pandas_profiling import ProfileReport
from numpy import sqrt

import matplotlib.pyplot as plt
plt.style.use("seaborn-deep")
import seaborn as sns

import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense, Input
from keras.utils import plot_model
import keras_tuner as kt

from scikeras.wrappers import KerasClassifier

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder, FunctionTransformer
from sklearn.metrics import mean_squared_error, mean_absolute_error, confusion_matrix, classification_report

<ipython-input-2-55e46eb7280f>:5: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport
<ipython-input-2-55e46eb7280f>:9: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use("seaborn-deep")


In [3]:
files.upload() # Cargamos el Kaggle.json (API)

!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list #Correr este comando verifica que está bien montado el driver de kaggle

Saving kaggle.json to kaggle.json
ref                                                              title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
---------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
lainguyn123/student-performance-factors                          Student Performance Factors                          94KB  2024-09-02 10:53:57           6288        140  1.0              
hanaksoy/customer-purchasing-behaviors                           Customer Purchasing Behaviors                         1KB  2024-09-01 22:18:07           3732         53  1.0              
haseebindata/student-performance-predictions                     Student Performance Predictions                       9KB  2024-08-17 06:57:57          11711        249  0.9411765        
computingvictor/2024-

In [4]:
!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.4.0+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 43.2/107.7 GB disk)


In [5]:
import pandas as pd
import numpy as np

import os
import os.path as osp
import random
import re
import shutil

#Visualización de datos
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

from IPython.display import display, Image

from numpy import asarray

from PIL import Image

from ultralytics import YOLO

In [6]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [7]:
my_seed = 19 #Semilla para reproducibilidad
IMG_EXT = ['.jpg', '.jpeg', '.png', '.bmp'] #Extensiones válidas de imagen
TEXT_EXT = '.txt'
YOLO_VERSION = 'yolov8'

In [8]:
random.seed(my_seed)

In [9]:
!kaggle datasets list

ref                                                              title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
---------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
lainguyn123/student-performance-factors                          Student Performance Factors                          94KB  2024-09-02 10:53:57           6288        140  1.0              
hanaksoy/customer-purchasing-behaviors                           Customer Purchasing Behaviors                         1KB  2024-09-01 22:18:07           3732         53  1.0              
haseebindata/student-performance-predictions                     Student Performance Predictions                       9KB  2024-08-17 06:57:57          11711        249  0.9411765        
computingvictor/2024-academic-ranking-of-world-universi

In [24]:
!kaggle datasets download hasibzunair/rsud20k-bangladesh-road-scene-understanding

Dataset URL: https://www.kaggle.com/datasets/hasibzunair/rsud20k-bangladesh-road-scene-understanding
License(s): Attribution-NonCommercial 4.0 International (CC BY-NC 4.0)
rsud20k-bangladesh-road-scene-understanding.zip: Skipping, found more recently modified local copy (use --force to force download)


In [26]:
ROOT_DIR = '/content'
DATASET_NAME = 'rsud20kbangladeshroadsceneunderstanding'
SUB_DATASET_NAME = 'RBRU.v1-yolov8'

In [27]:
print(f"!unzip {DATASET_NAME}.zip -d {ROOT_DIR}/{DATASET_NAME}")

!unzip rsud20kbangladeshroadsceneunderstanding.zip -d /content/rsud20kbangladeshroadsceneunderstanding


In [28]:
%cd {ROOT_DIR}
!mkdir /{DATASET_NAME}
!unzip {DATASET_NAME}.zip -d {ROOT_DIR}/{DATASET_NAME}

/content
mkdir: cannot create directory ‘/rsud20kbangladeshroadsceneunderstanding’: File exists
unzip:  cannot find or open rsud20kbangladeshroadsceneunderstanding.zip, rsud20kbangladeshroadsceneunderstanding.zip.zip or rsud20kbangladeshroadsceneunderstanding.zip.ZIP.


In [29]:
DATA_DIR = f"{ROOT_DIR}/{DATASET_NAME}/{SUB_DATASET_NAME}"
print(DATA_DIR)

/content/rsud20kbangladeshroadsceneunderstanding/RBRU.v1-yolov8


In [30]:
subdirs = ['train', 'test', 'valid']

In [31]:
train_folder = osp.join(DATA_DIR, subdirs[0])
train_img_folder = osp.join(train_folder, "images")
train_lbl_folder = osp.join(train_folder, "labels")
val_folder = osp.join(DATA_DIR, subdirs[2])
val_img_folder = osp.join(val_folder, "images")
val_lbl_folder = osp.join(val_folder, "labels")
test_folder = osp.join(DATA_DIR, subdirs[1])
test_img_folder = osp.join(test_folder, "images")
test_lbl_folder = osp.join(test_folder, "labels")

In [17]:
folder_list = [train_img_folder, train_lbl_folder,
                val_img_folder, val_lbl_folder,
                test_img_folder, test_lbl_folder
]

print("Imprimimos las rutas de las carpetas:")
for fa in folder_list:
  print(fa)

Imprimimos las rutas de las carpetas:
/content/rsud20kbangladeshroadsceneunderstanding/RBRU.v1-yolov8/train/images
/content/rsud20kbangladeshroadsceneunderstanding/RBRU.v1-yolov8/train/labels
/content/rsud20kbangladeshroadsceneunderstanding/RBRU.v1-yolov8/valid/images
/content/rsud20kbangladeshroadsceneunderstanding/RBRU.v1-yolov8/valid/labels
/content/rsud20kbangladeshroadsceneunderstanding/RBRU.v1-yolov8/test/images
/content/rsud20kbangladeshroadsceneunderstanding/RBRU.v1-yolov8/test/labels


In [18]:
folder_list = [train_img_folder, val_img_folder,
               test_img_folder]

In [19]:
print(train_img_folder)

/content/rsud20kbangladeshroadsceneunderstanding/RBRU.v1-yolov8/train/images


In [20]:
img_name_list = []
img_width_list = []
img_height_list = []
img_folder_list = []
img_format_list = []

for fa in folder_list:
  parent = os.path.dirname(fa)
  parent_name = os.path.basename(parent)
  for fname in os.listdir(fa):
    img_folder_list.append(parent_name)
    img_name_list.append(fname)
    img_path = osp.join(fa, fname)
    img_format_list = "ok" if osp.splitext(fname)[-1] in IMG_EXT else "not ok"

    with Image.open(img_path) as img:
      width, height = img.size
      img_width_list.append(width)
      img_height_list.append(height)

data_model = {"folder": img_folder_list, "image_name": img_name_list,
              "width": img_width_list, "height": img_height_list
              , "format": img_format_list}
report_df = pd.DataFrame(data=data_model)

FileNotFoundError: [Errno 2] No such file or directory: '/content/rsud20kbangladeshroadsceneunderstanding/RBRU.v1-yolov8/train/images'

In [ ]:
IMAGE_SIZE = 640

In [ ]:
def show_img_matrix(image_list, show_axis=False):
  MAX_SIZE = 9
  if len(image_list) != MAX_SIZE:
    return

  plt.figure(figsize=(10, 10))

  for i in range(MAX_SIZE):
    img_file = image_list[i]
    img_name = osp.basename(img_file)
    ax = plt.subplot(3, 3, i + 1)
    if osp.exists(img_file) == True:
      plt.imshow(asarray(Image.open(img_file)))
      img = mpimg.imread(img_file)
    #Como los nombres son muy extensos, vamos a truncar a 20 caracteres.
    plt.title(img_name[:20])
    plt.axis("on" if show_axis == True else "on")

  plt.tight_layout()
  plt.show()

In [ ]:
img_list = [filename for filename in os.listdir(train_img_folder)]
img_count = len(img_list)
print(f"El número total de imágenes es: {img_count}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
class_labels = {
    0: 'person',
    1: 'rickshaw',
    2: 'rickshaw van',
    3: 'auto rickshaw',
    4: 'truck',
    5: 'pickup truck',
    6: 'private car',
    7: 'motorcycle',
    8: 'bicycle',
    9: 'bus',
    10: 'micro bus',
    11: 'covered van',
    12: 'human hauler'
}